# 資料合併-長程

In [5]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/long'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/sydney.csv',
    f'{base_path}/sydney_business.csv',
    f'{base_path}/zurich.csv',
    f'{base_path}/zurich_business.csv',
    f'{base_path}/losangeles.csv',
    f'{base_path}/losangeles_business.csv',
    f'{base_path}/frankfurt.csv',
    f'{base_path}/frankfurt_business.csv',
    f'{base_path}/france.csv',
    f'{base_path}/newyork.csv',
    f'{base_path}/newyork_business.csv',
    f'{base_path}/london.csv',
    f'{base_path}/london_business.csv',
]

# 合併檔案
merge_files(file_paths, output_path)

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/sydney.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/sydney_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/zurich.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/zurich_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/losangeles.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/losangeles_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/frankfurt.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/frankfurt_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/france.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/newyork.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/newyork_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/london.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/l

# 長程航班

In [7]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# Helper Functions
# --------------------------
def move_column(df, col_to_move, ref_col, position="after"):
    """
    將 col_to_move 移至 ref_col 之前或之後
    position: "before" 或 "after"
    """
    cols = df.columns.tolist()
    if col_to_move in cols and ref_col in cols:
        cols.remove(col_to_move)
        ref_idx = cols.index(ref_col)
        insert_idx = ref_idx if position == "before" else ref_idx + 1
        cols.insert(insert_idx, col_to_move)
    return df[cols]

def convert_date(series):
    """統一解析日期格式"""
    dt = pd.to_datetime(series, format="%Y-%m-%d", errors="coerce")
    dt = dt.combine_first(pd.to_datetime(series, format="%Y/%m/%d", errors="coerce"))
    return dt.dt.strftime("%Y-%m-%d")

def convert_time_to_minutes(time_str):
    """
    將格式如 'x 小時 y 分鐘' 或只有小時部分轉換為分鐘數
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return np.nan

def parse_duration(duration_str):
    """
    與 convert_time_to_minutes 相似，用於解析「第一段飛行時間」與「第二段飛行時間」
    """
    return convert_time_to_minutes(duration_str)

# --------------------------
# 1. 載入資料及初步處理
# --------------------------
data_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data = pd.read_csv(data_path, low_memory=False)
data = data.drop(columns=["轉機點"], errors="ignore")

# 統一「出發日期」格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 2. 計算對數變數與新增最低價格天數
# --------------------------
data["平均價格"] = pd.to_numeric(data["平均價格"], errors='coerce')
data["最低價格"] = pd.to_numeric(data["最低價格"], errors='coerce')
data["價格變異"] = pd.to_numeric(data["價格變異"], errors='coerce')
data["中位數價格"] = pd.to_numeric(data["中位數價格"], errors='coerce')
data["最低價格剩餘天數"] = pd.to_numeric(data["最低價格剩餘天數"], errors='coerce')

data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)

# 新增最低價格天數 (91 - '最低價格剩餘天數')
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除關鍵欄位有 NaN 的資料
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 3. 假期判斷
# --------------------------
# 定義各國假期範圍
holidays = {
    "台灣": [("2025-01-20", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "美國": [("2025-01-18", "2025-01-20"), ("2025-02-15", "2025-02-17")],
    "澳洲": [("2025-01-25", "2025-01-27")],
    "瑞士": [("2025-01-29", "2025-01-30")]
}

# 抵達機場代號與地區對應
airport_to_region = {
    "LAX": "美國",  # 洛杉磯
    "SYD": "澳洲",  # 雪梨
    "FRA": "德國",  # 法蘭克福
    "CDG": "法國",  # 巴黎
    "JFK": "美國",  # 紐約
    "LHR": "英國",  # 倫敦
    "ZRH": "瑞士"   # 蘇黎世
}

# 轉換「出發日期」為 datetime 格式（方便比較）
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期（不論目的地，出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 依據抵達機場對應的地區判斷其他國家假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 將「假期」欄位移至「艙等」欄位之後（若存在「艙等」欄位）
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 4. 合併額外地區經濟資料
# --------------------------
additional_data = {
    "Region": ["美國", "澳洲", "德國", "英國", "法國", "瑞士"],
    "Cost of Living Index": [70.4, 70.2, 62.2, 62.0, 63.7, 101.1],
    "GDP (PPP) per capita (in thousand USD)": [82.715, 67.901, 69.532, 60.735, 63.881, 93.055],
    "經濟指標": [0.5468126, -0.389354, -0.7021, -1.262158, -0.977473, 2.7842724]
}
region_df = pd.DataFrame(additional_data)

# 利用抵達機場代號映射對應地區
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 5. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 6. 新增機場分類欄位
# --------------------------
super_large_airports = ['LHR', 'LAX']
large_airports = ['FRA', 'CDG', 'JFK'] 
medium_large_airports = ['SYD', 'ZRH']

data['機場分類'] = data['抵達機場代號'].apply(
    lambda x: 1 if x in super_large_airports else (2 if x in medium_large_airports else None)
).astype('Int64')

# 將「機場客運量分類」移至「抵達機場代號」後
cols = data.columns.tolist()
if '抵達機場代號' in cols and '機場分類' in cols:
    arrival_idx = cols.index('抵達機場代號')
    cols.insert(arrival_idx + 1, cols.pop(cols.index('機場分類')))
data = data[cols]

# --------------------------
# 7. 新增機場指標欄位
# --------------------------
airport_index = {
    "LAX": 1.7011215,
    "JFK": 0.5891658,
    "LHR": 1.0504261,
    "CDR": 0.5969613,
    "FRA": 0.1276154,
    "SYD": -1.56618,
    "ZRH": -2.49911
}

# 新增「機場指標」欄位
data['機場指標'] = data['抵達機場代號'].map(airport_index)

# 將「機場指標」移至「經濟指標」欄位之後
cols = data.columns.tolist()
if '經濟指標' in cols and '機場指標' in cols:
    flight_idx = cols.index('經濟指標')
    cols.insert(flight_idx + 1, cols.pop(cols.index('機場指標')))
data = data[cols]

# --------------------------
# 8. 新增「飛行時間_分鐘」與「停留時間_分鐘」欄位
# --------------------------
def convert_time_to_minutes(time_str):
    """
    將格式如 'x 小時 y 分鐘' 轉換為分鐘數。
    若只有小時部分，也能正確解析。
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

# 新增飛行時間及停留時間轉換結果
data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_time_to_minutes)
data['停留時間_分鐘'] = data['停留時間'].apply(convert_time_to_minutes)

data['飛行時間_分鐘'] = data['飛行時間_分鐘'].fillna(0).astype('int64')
data['停留時間_分鐘'] = data['停留時間_分鐘'].fillna(0).astype('int64')

# 將「飛行時間_分鐘」移至「飛行時間」後面
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
    
# 將「停留時間_分鐘」移至「停留時間」後面
if '停留時間' in cols and '停留時間_分鐘' in cols:
    layover_idx = cols.index('停留時間')
    cols.insert(layover_idx + 1, cols.pop(cols.index('停留時間_分鐘')))
data = data[cols]

# --------------------------
# 9. 轉換航空公司組合及航空聯盟欄位型態
# --------------------------
if '航空公司組合' in data.columns:
    data["航空公司組合"] = pd.to_numeric(data["航空公司組合"], errors='coerce').round().astype('Int64')
if '航空聯盟' in data.columns:
    data["航空聯盟"] = pd.to_numeric(data["航空聯盟"], errors='coerce').round().astype('Int64')
    
# --------------------------
# 10. 新增競爭航班數欄位
# --------------------------
# 為每筆航班標記其群組數量
data['competing_flights'] = data.groupby(
    ['出發機場代號', '抵達機場代號', '出發日期', '出發時段']
)['航空公司'].transform('size')

data = data.sort_values(
    by=['出發機場代號', '抵達機場代號', '出發日期', '出發時段', '航空公司']
)

# --------------------------
# 11. 新增「飛行時間兩段分類」欄位
# --------------------------
def classify_flight_segments(row):
    # 若直飛（停靠站數量為 0），直接回傳 0
    if row['停靠站數量'] == 0:
        return 0
    try:
        first_duration = parse_duration(row["第一段飛行時間"])
        second_duration = parse_duration(row["第二段飛行時間"])
        if first_duration > second_duration:
            return 1
        elif first_duration < second_duration:
            return 2
        else:
            return 1  # 相等則預設 1
    except Exception:
        return np.nan

# 假設資料中有「第一段飛行時間」與「第二段飛行時間」欄位
data['飛行時間兩段分類'] = data.apply(classify_flight_segments, axis=1)
# 如果有 NaN 值，視需求決定要填入哪個數字，這裡範例填入 0
data['飛行時間兩段分類'] = data['飛行時間兩段分類'].fillna(0).astype(int)

data = move_column(data, "飛行時間兩段分類", "第二段飛行時間", position="after")

# --------------------------
# 12. 新增艙等分類欄位 
# --------------------------
if "艙等" in data.columns:
    def classify_cabin(cabin_class):
        # 分割艙等資訊
        cabins = cabin_class.split(" ") if isinstance(cabin_class, str) else []
        if len(cabins) == 2:
            if cabins[0] == "經濟艙" and cabins[1] == "經濟艙":
                return "1"
            elif cabins[0] == "經濟艙" and cabins[1] != "經濟艙":
                return "2"
            elif cabins[0] != "經濟艙" and cabins[1] == "經濟艙":
                return "3"
            elif cabins[0] != "經濟艙" and cabins[1] != "經濟艙":
                return "4"
        return None

# 新增「艙等分類」欄位
data["艙等分類"] = data["艙等"].apply(classify_cabin)

# 將「艙等分類」移至「艙等」後
data = move_column(data, "艙等分類", "艙等", position="after")

# --------------------------
# 13. 匯出處理後的資料
# --------------------------
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_final.csv'
data.to_csv(output_path, index=False)

抵達機場代號
LHR    17113
LAX    16878
FRA    13670
SYD    11178
CDG    11079
ZRH     9741
JFK     9153
Name: count, dtype: int64
